In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Network

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        

In [10]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,2)),
            
            nn.Conv2d(6, 16, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2,2)),
        )
        
        
        self.fc = nn.Sequential(
        nn.Linear(16*5*5, 120),
        nn.ReLU(inplace=True),
        nn.Linear(120, 84),
        nn.ReLU(inplace=True),
        nn.Linear(84, 10),
        )
        
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1) # can't use nn.Flatten(x, 1)
        x = self.fc(x)
        return x

In [11]:
net = LeNet()

# Train

In [12]:
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output)

tensor([[-0.0213,  0.0181, -0.0198, -0.0404, -0.0399, -0.0320, -0.0041, -0.0535,
          0.1362,  0.0119]], grad_fn=<AddmmBackward0>)


In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

epoch = 10
for i in range(epoch):
    optimizer.zero_grad()
    
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step() 